OpenAI

In [1]:
!pip install openai==0.28

import openai
from google.colab import userdata

# APIキーを取得
api_key = userdata.get("GPT2_API_KEY")
openai.api_key = api_key  # ここにあなたのAPIキーを設定してください

def encode_text(text):
    response = openai.Embedding.create(
        input=text,
        model="text-embedding-3-large"  # または "text-embedding-3-small"
    )
    return response["data"][0]["embedding"]

text1 = "全国的に高気圧に覆われ晴れたが、午後は気圧の谷や湿った空気の影響で西・北日本は曇りや雨。熱帯低気圧に近い沖縄・奄美は雨。北海道浜鬼志別 20.5mm/1h は 10 月 1 位更新。"
text2 = "2022年1月14日、各地で寒波が続き、特に北海道地方は強い風と雪に見舞われました。札幌や網走では積雪が10cm以上で、風速も最大17.4m/sに達しています。各地で厳しい冬の気象条件が報告されました。"

embedding1 = encode_text(text1)
embedding2 = encode_text(text2)

import numpy as np

def cosine_similarity(embedding1, embedding2):
    return np.dot(embedding1, embedding2) / (np.linalg.norm(embedding1) * np.linalg.norm(embedding2))

similarity_score = cosine_similarity(embedding1, embedding2)
print(f"類似度スコア: {similarity_score}")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 2.9 MB/s eta 0:00:00
類似度スコア: 0.4406855280834369


In [16]:
!pip install openai==0.28

import openai
# 1. Google Driveのマウント
from google.colab import drive
drive.mount('/content/drive')
from google.colab import userdata

# APIキーを取得
api_key = userdata.get("GPT2_API_KEY")
openai.api_key = api_key  # ここにあなたのAPIキーを設定してください

# 3. テキストファイルを読み込むための関数
def read_text_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return file.read()

def encode_text(text):
    response = openai.Embedding.create(
        input=text,
        model="text-embedding-3-large"  # または "text-embedding-3-small"
    )
    return response["data"][0]["embedding"]

text1 = read_text_file(f'/content/drive/My Drive/研究用/2023_12_調査ノート/訂正用/コメント/元のコメント/2022_01_14_original.txt')
text2 = read_text_file(f'/content/drive/My Drive/研究用/2023_12_調査ノート/訂正用/コメント/生成したコメント/v4_gpt4o_生成されたコメント/2022_01_14_gpt4o_generate_v4.txt')

embedding1 = encode_text(text1)
embedding2 = encode_text(text2)

import numpy as np

def cosine_similarity(embedding1, embedding2):
    return np.dot(embedding1, embedding2) / (np.linalg.norm(embedding1) * np.linalg.norm(embedding2))

similarity_score = cosine_similarity(embedding1, embedding2)
print(f"類似度スコア: {similarity_score}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
類似度スコア: 0.7278830176812094


In [5]:
!pip install openai==0.28 janome nltk rouge

import openai
from google.colab import userdata
from janome.tokenizer import Tokenizer
import numpy as np
from nltk.translate.bleu_score import sentence_bleu
from rouge import Rouge

# APIキーを取得
api_key = userdata.get("GPT2_API_KEY")
openai.api_key = api_key  # ここにあなたのAPIキーを設定してください

# 日本語のテキストのトークン化
def tokenize_japanese(text):
    tokenizer = Tokenizer()
    return [token.surface for token in tokenizer.tokenize(text)]

def encode_text(text):
    # テキストをトークン化してから埋め込みを生成
    tokenized_text = " ".join(tokenize_japanese(text))
    response = openai.Embedding.create(
        input=tokenized_text,
        model="text-embedding-3-large"  # または "text-embedding-3-small"
    )
    return response["data"][0]["embedding"]

def cosine_similarity(embedding1, embedding2):
    return np.dot(embedding1, embedding2) / (np.linalg.norm(embedding1) * np.linalg.norm(embedding2))

def calculate_bleu(reference, candidate):
    return sentence_bleu([reference], candidate)

def calculate_rouge(reference, candidate):
    rouge = Rouge()
    scores = rouge.get_scores(" ".join(candidate), " ".join(reference))
    return scores[0]['rouge-1']['f']

text1 = "全国的に高気圧に覆われ晴れたが、午後は気圧の谷や湿った空気の影響で西・北日本は曇りや雨。熱帯低気圧に近い沖縄・奄美は雨。北海道浜鬼志別 20.5mm/1h は 10 月 1 位更新。"
text2 = "日本付近は高気圧に覆われ、概ね晴れ。沖縄地方は低気圧の影響で雨が多く、風も強い。那覇で最大瞬間風速 16.6m/s、宮古島で合計降水量 19.0mm。"

# Embedding類似度の計算
embedding1 = encode_text(text1)
embedding2 = encode_text(text2)
similarity_score = cosine_similarity(embedding1, embedding2)

# BLEUとROUGE-1スコアの計算
tokenized_text1 = tokenize_japanese(text1)
tokenized_text2 = tokenize_japanese(text2)
bleu_score = calculate_bleu(tokenized_text1, tokenized_text2)
rouge_score = calculate_rouge(tokenized_text1, tokenized_text2)

print(f"Embedding類似度スコア: {similarity_score}")
print(f"BLEUスコア: {bleu_score}")
print(f"ROUGE-1スコア: {rouge_score}")

Embedding類似度スコア: 0.67202385276731
BLEUスコア: 0.07420116852932022
ROUGE-1スコア: 0.43373493476556835
